# Assignment 8: Space charge map implementation

<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li>Implement the space charge map in drift and quadrupole.</li>
            <li>Track the beam through a FODO cell with & without space charge considered.</li>
            <li>Compare the beam envelope.</li>
            <li>Check the stability criterion.</li>
            </ul>
    </div>
</html>

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>If you use google colab, run this cell:</strong>
    </div>
</html>

In [ ]:
# for google colab, run this cell
!git clone https://github.com/potato18z/pam1-hs2021.git
import sys
sys.path.append('./pam1-hs2021')

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>If you run it locally, run</strong>
               </div>
</html>

```bash
$ cd .../pam1-hs2021
...pam1-hs2021$ git pull
```
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
to get the updated repository.</div></html>

## Initialization
<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li> Simply run this cell to instantiante an `Proton` species and set all global variables in the `Parameter` module. Take 3 MeV as the kinetic energy. </li>
        </ul>
    </div>
</html>

In [ ]:
import numpy as np
import AcceLEGOrator.Parameter as param
from AcceLEGOrator import Proton, Constants, Physics
from AcceLEGOrator import Gaussian, Bunch
import matplotlib.pyplot as plt
import copy

c = Constants.clight

Ekin = 3 # MeV

proton = Proton()

# set global variables
param.gamma_0 = Physics.getGamma(Ekin, proton.mass)
param.mass = proton.mass # MeV / c^2
param.charge = proton.charge # e

beta0 = Physics.getBeta(param.gamma_0)
p0 = param.gamma_0 * beta0 * param.mass   # MeV / c
Brho = p0 / param.charge * 1e6 / c # T*m

print('gamma =', param.gamma_0)
print('beta  =', beta0)
print('p0    =', p0 , 'MeV/c')
print('Br    =', Brho, 'Tm')

<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li> Simply run this cell to define the test FODO cell.</li>
        </ul>
    </div>
</html>

![fodo_for_sc](img/fodo_for_sc.png)

In [ ]:
from AcceLEGOrator import Drift, Quadrupole
# quadrupole length [m]
Lq = 0.05

# quandrupole strength [1/m^2]
kq = 400.0

# magnetic field gradient [T/m]
gradBf = kq * Brho

# half focusing quadrupole
Qf = Quadrupole(Lq, gradBf)

print('focusing Quadrupole:', Qf)

# magnetic field gradient [T/m]
gradBd = -kq * Brho

# defocusing quadrupole
Qd = Quadrupole(Lq, gradBd)

print('defocusing Quadrupole:', Qd)

# drift length [m]
Ldr = 0.05

# drift
Dr_h = Drift(Ldr/2.) # half drift
Dr = Drift(Ldr)

print('half drift:', Dr_h)
print('drift:', Dr)

# define FODO cell
fodo = [Dr_h, Qf, Dr, Qd, Dr_h]

# collect fodo parameters together
fodo_params = {'Lq':Lq, 
               'gradBf': gradBf,
               'gradBd': gradBd,
               'Ldr': Ldr}

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Use the `getMatchedSigma` function in `Tracking.py` to calculate the the sigma matrix that is matched to the fodo cell. We assume an rms emittance of $\epsilon_{rms,x}=\epsilon_{rms,y}=0.3 [mm\cdot mrad] = 0.3 \cdot 10^{-6}[m\cdot rad]$ </li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Tracking

emit = 0.3/(beta0*param.gamma_0)*1e-6 # m*rad
sigma_matched = Tracking.getMatchedSigma(...)

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
           <ul>
            <li>Use the `trackSigma` function in `Tracking.py` to track the sigma matrix for $n_{cells}=4$. </li>
            <li>Plot the rms beam sizes in x and y. The parameter `n_slice` cuts each element (except drift) into shorter slices to get a smoother output. Change $n_{slice}$ from 1 and see the difference. The result should be quite ok for $n_{slice}> 5$.</li>
            <li>Check if you can reproduce the plot on Page 22 of `pam-s8.pdf`.</li>
        </ul>
    </div>
</html>

You can check the `_sliceMap()` function in `Tracking.py` to see how `get()` function works.

In [ ]:
all_sigmas_matched, lengths = Tracking.trackSigma(sigma_matched, cell=fodo, n_cells=4, n_slice=...)

# plot
fig = plt.figure()
fig.set_size_inches(6,6)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel('RMS beam size [mm]')

axis.plot(lengths, ..., label=r'$\langle x\rangle [mm]$', color='black')
# note that the original unit is [m] and here we want [mm]
axis.plot(lengths, ..., label=r'$\langle y\rangle [mm]$', color='red')

axis.legend(loc='upper left')
plt.title('Mathched beam envelope without space charge')
plt.show()

## Space charge map - general solution for various elements
From equation (23) of `pam-s7` we can derive that for a drift
$$
x'' - \frac{K}{r_0^2}x = 0,\quad\quad y''- \frac{K}{r_0^2}y = 0.
$$
where $(\cdot)' = \frac{d(\cdot)}{ds}$, perveance $K = \frac{qI}{2\pi\varepsilon_0 m_0 c^3\beta^3\gamma^3}$ and $r_0$ is the radius of a uniformly distributed circular beam.

From equation (24) of `pam-s7` we know that for a quadrupole focusing in $x$ direction
$$
\begin{align}
x'' + k_1x - \frac{K}{r_0^2}x &= 0,\\
y'' - k_1y - \frac{K}{r_0^2}y &= 0.
\end{align}
$$
where $k_1 = \frac{q}{P_0}\cdot gradB$ is the normalised quadrupole gradient.

We can write the above equations into a generalised form
$$
u''(s) + C\cdot u(s) = 0
$$
where $u=x$ or $y$, and C takes different values according to different elements.


<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li> Check that we can write a general solution to the above ODE.</li>
$$
u(s) = c_1\cdot\cos\left(\sqrt{C}s\right) + c_2\cdot\sin\left(-\sqrt{C}s\right)
$$
where $C$ could be negative and $\sqrt{C}$ could be complex.
        </ul>
    </div>
</html>

Note: You can direly write your proof in latex here, or write it in paper and take a picture as before.

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Derive the linear map of space charge for an element of length $L$.
            <li>in horizontal directions (i.e. a $4\times 4$ map). In longitudinal direction we neglect the space charge force.</li>
        </ul>
    </div>
</html>

$$
\begin{align}
u(0) &= ... \\
u'(0) &= ... \\
u(L) &= ...\\
u'(L) &= ...
\end{align}
$$

In matrix format
$$
\begin{pmatrix}u \\ u'\end{pmatrix}^L =
\begin{pmatrix}... & ... \\ ... & ... \end{pmatrix}\cdot
\begin{pmatrix}u \\ u'\end{pmatrix}^0.
$$

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Define the space charge map for drift and quadrupole using the previous result in the horizontal directions. In longitudinal direction we neglect the space charge force.</li>
        </ul>
    </div>
</html>

#### Notes
- You can get inspiration from the original definition of Drift and Quadrupole maps in `Map.py`.
- When calculating $I_0$, keep in mind that our proton.mass already has the unit of $MeV/c^2$.
- `np.lib.scimath.sqrt` is used here instead of the usual `np.sqrt` to handle complex cases. Pay close attention to the sign of $C$.
- In the case of a uniformly distributed circular beam of radius $r_0$, there is the beam area $\pi r_0^2$ inside the denominator of the space charge term. Since our beam is
    - Gaussian,
    - although it starts from a 'circle' in $x$ and $y$, it grows to become ellipses during the evolution
  
  We need to find a proper way to estimate the action area of space charge. Since the area of ellipse is $\pi ab$,  we simply use here $\sqrt{\langle x^2\rangle}$ and $\sqrt{\langle y^2\rangle}$ to approximate $a$ and $b$, i.e. $r_0^2 \sim \sqrt{\langle x^2\rangle\cdot \langle y^2\rangle}$.

In [ ]:
from AcceLEGOrator import Map

class SpaceChargeDrift(Map):
    
    # param length: element length [m]
    # param sigma: the sigma matrix of the bunch being tracked
    # param I_sc: beam current for evaluating space charge [A]
    def __init__(self, length, sigma, I_sc):
        self.length = length
        self.__sigma = sigma
        beta_0 = Physics.getBeta(param.gamma_0)
       
        eps0 = Constants.epsilon0
        c    = Constants.clight
        e    = Constants.echarge
        q    = param.charge * e # [C]

        self.I = I_sc
        self.I0 = ... # unit [A], page 14 pam-s7, param.mass has unit MeV/c^2
        self.K = ... # equation (22), pam-s7
        
        r02 = np.sqrt(...) # use the product of rms beam sizes to estimate beam area
        
        C = ...

        f = length / (beta_0 * param.gamma_0) ** 2
        
        R = np.real_if_close(np.matrix(
            [[...,     ...,    0,      0,      0,  0],
            [...,      ...,    0,      0,      0,  0],
            [0,        0,      ...,    ...,    0,  0],
            [0,        0,      ...,    ...,    0,  0],
            [0,        0,      0,      0,      1,  f],
            [0,        0,      0,      0,      0,  1]], dtype = 'complex_'))
        super(SpaceChargeDrift, self).__init__(R, length)
    
    
    def __str__(self):
        return 'SpaceChargeDrift(I = ' + str(self.I) + ' [A],\n'\
               + '\t\t      I_0 = ' + str(self.I0) + '[A])\n'\
               + '\t\t      K = ' + str(self.K) + ')\n'\
               + '\t\t      length = ' + str(self.length) + '[m]'
    
    def get(self, length, sigma=None):
        # return another SpaceChargeDrift with different sigma and/or length
        if sigma is None:
            return SpaceChargeDrift(length, self.__sigma, self.I)
        else:
            return SpaceChargeDrift(length, sigma, self.I)

In [ ]:
class SpaceChargeQuadrupole(Map):
    
    # param length in [m]
    # param gradB in [T/m]
    # param sigma: the sigma matrix of the bunch being tracked
    # param I_sc: beam current for evaluating space charge [A]
    def __init__(self, length, gradB, sigma, I_sc):
        self.length = length
        self.__gradB = gradB
        self.__sigma = sigma
        beta_0 = Physics.getBeta(param.gamma_0)
        
        eps0 = Constants.epsilon0
        c    = Constants.clight
        e    = Constants.echarge
        q    = param.charge * e # [C]
        
        P_0 = param.mass * 1e6 / c * param.gamma_0 * beta_0
        k_1 = param.charge / P_0 * self.__gradB
        
        self.I = I_sc
        self.I0 = ... # unit [A], page 14 pam-s7, param.mass has unit MeV/c^2
        self.K = ... # equation (22), pam-s7
        
        r02 = np.sqrt(...)
        
        C = ...
        
        f = length / (beta_0 * param.gamma_0) ** 2
        
        R = np.real_if_close(np.matrix(
            [[...,     ...,    0,      0,      0,  0],
            [...,      ...,    0,      0,      0,  0],
            [0,        0,      ...,    ...,    0,  0],
            [0,        0,      ...,    ...,    0,  0],
            [0,        0,      0,      0,      1,  f],
            [0,        0,      0,      0,      0,  1]], dtype = 'complex_'))
        super(SpaceChargeQuadrupole, self).__init__(R, length)
    
    
    def __str__(self):
        return 'SpaceChargeQuadrupole(I = ' + str(self.I) + ' [A],\n'\
               + '\t\t      I_0 = ' + str(self.I_0) + '[A])\n'\
               + '\t\t      K = ' + str(self.K) + ')\n'\
               + '\t\t      gradB = ' + str(self.__gradB) + '[T/m]\n'\
               + '\t\t      length = ' + str(self.length) + '[m]'

    def get(self, length, sigma=None):
        # return another SpaceChargeQuadrupole with different sigma and/or length
        if sigma is None:
            return SpaceChargeQuadrupole(length, self.__gradB, self.__sigma, self.I)
        else:
            return SpaceChargeQuadrupole(length, self.__gradB, sigma, self.I)

## Tracking through space charge incorporated FODO cells
<html>
    <div class="alert alert-info" role="alert" style="margin-top: 10px">
        <ul>
            <li> Simply run this cell to define space charge incorporated FODO cell.</li>
        </ul>
    </div>
</html>

In [ ]:
def get_fodo_sc(fodo_params, sigma, I):  
    # FODO parameters
    Lq = fodo_params['Lq']
    gradBf = fodo_params['gradBf']
    gradBd = fodo_params['gradBd']
    Ldr = fodo_params['Ldr']

    # define the space charge FODO cell
    # focusing quadrupole
    Qf_sc = SpaceChargeQuadrupole(Lq, gradBf, sigma, I)

    # defocusing quadrupole
    Qd_sc = SpaceChargeQuadrupole(Lq, gradBd, sigma, I)

    # drift
    Dr_h_sc = SpaceChargeDrift(Ldr/2., sigma, I) # half drift
    Dr_sc = SpaceChargeDrift(Ldr, sigma, I)

    # define space charge FODO cell
    fodo_sc = [Dr_h_sc, Qf_sc, Dr_sc, Qd_sc, Dr_h_sc]
    return fodo_sc

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Define the following function to track sigma through space charge incorporated FODO beamline.</li>
        </ul>
    </div>
</html>

#### Notes
- Since sigma is changing throughout tracking, we need to update sigma and the accelerator elements together. For example, to track sigma through [drift, quadrupole]
    - progagate sigma through drift;
    - collect the sigma after drift;
    - use the new sigma to define a new quadrupole; **Use the get() function here**
    - propagate sigma through the new quadrupole
    - collect the sigma after quadrupole.
  
  Note that the above process should loop over each **slice** of acclerator element. Each slice of M has length $M.length/n_{slice}$.

In [ ]:
def track_fodo_sc(fodo_sc, sigma, n_cells=1, n_slice=1):
    # initial sigma
    sigmas = np.expand_dims(sigma, axis=0)
    
    # initial length
    length_till_now = 0.
    lengths = [0,]

    cells = {} # this is to collect the updated FODO cells
    for n in range(n_cells):
        cells[n] = []
        for M in fodo_sc:
            L = M.length
            for i in range(n_slice):
                # use the current sigma to define slice i map
                M_slice_i = M.get(...)

                # calculate length
                length_till_now = length_till_now + M_slice_i.length
                lengths.append(length_till_now)

                # propagate sigma using M_slice_i
                sigma = ...

                # collect sigma
                sigmas = np.append(sigmas, np.expand_dims(sigma, axis=0), axis=0)

                # collect maps
                cells[n].append(M_slice_i)
    return sigmas, lengths, cells

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Track the sigma matrix through the space charge incorprated FODO cell for different beam current $I=0mA$, $60mA$ and $300mA$. Compare your result with page 23 of `pam-s8`.</li>
        </ul>
    </div>
</html>

In [ ]:
fodo_sc = get_fodo_sc(fodo_params, sigma_matched, I=...) # I has unit [A]
all_sigmas_sc, lengths_sc, cells_sc = track_fodo_sc(fodo_sc, sigma_matched, n_cells=4, n_slice=5)
# plot
fig = plt.figure()
fig.set_size_inches(6,6)
axis = fig.add_subplot(111)
axis.set_xlabel('s [m]')
axis.set_ylabel('RMS beam size [mm]')

axis.plot(lengths_sc, ..., label=r'$\langle x\rangle [mm]$', color='black')
axis.plot(lengths_sc, ..., label=r'$\langle y\rangle [mm]$', color='red')

axis.legend(loc='upper left')
plt.title('Beam envelope with space charge I=...mA')
plt.show()

## Stability analysis

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Based on the BIBO stability criterion given on Page 85 of `pam-s5`, what is the stability limit, i.e. the maximal current ($I_max$) such that the beam is still stable?</li>
        </ul>
    </div>
</html>

In [ ]:
cell = cells_sc[3]#fetch the updated FODO_sc cell here. Since we track 4 cells, the indices you can take are 0,1,2,3
M_fodo_sc = Tracking.getMcell(cell=cell)
# calculate stability
#...

## Bonus (Open questions)
- Besides Gaussian, define a elliptical bunch following uniform distribution. Instead of simply drawing the ellipse from parameters $a$ and $b$, we need to define the elliptical bunch according to its sigma matrix.
  
  Thoughts: In 1-dimensional case, if we use sigma[0,0] as var(x) and 0 as mean(x), we would get the lower and upper limits of uniform distribution: $[-\sqrt{3\sigma_{0,0}}, \sqrt{3\sigma_{0,0}}]$. But if we do this for all 6 dimensions, we would get a cube instead of an ellipse. Discarding particles outside the ellipse would then change the number of particles and also the sigma, not to mention that we still don't know the envelope of our desired ellipse yet.
  
- If an uniformly distributed ellipse is successfully defined, we can use equation (21) of [1] to calculate the space charge.

## References
- [1] K. Schindl, Space Charge, https://cds.cern.ch/record/941316/files/p305.pdf
- [2] L.Hein, CERN Linac4 - The Space Charge Challenge, https://cds.cern.ch/record/1626566
- [3] B.Holzer, Lattice Design in Particle Accelerators II, https://indico.cern.ch/event/103406/contributions/20657/attachments/11961/17473/CAS_Holzer_II.pdf
- [4] Emittance Diagnostics, https://cas.web.cern.ch/sites/default/files/lectures/dourdan-2008/braun-emittance.pdf